In [1]:
from dicesapi import DicesAPI
from dicesapi.text import CtsAPI, Passage
from dicesapi.jupyter import NotebookPBar
import pandas as pd

In [2]:
api = DicesAPI(
    dices_api='http://localhost:8000/api',
    logfile='ombretta.log'
)
cts = CtsAPI(dices_api=api)

In [3]:
speeches = api.getSpeeches(author_name='Homer')

#### Make two small adjustments

In [7]:
# cludge for textual variant in Odyssey
drop = []

for s in speeches:
    if s.work.title == 'Odyssey':
        if s.l_fi == '10.456':
            s.l_fi = '10.457'
        if s.l_fi.split('.')[0] != s.l_la.split('.')[0]:
            drop.append(s)
            
# another cludge to remove the apologia
speeches = sorted(speeches - drop)


TypeError: 'NoneType' object is not iterable

### Get the text

In [9]:
pbar = NotebookPBar(max=len(speeches))
for s in (speeches):
    pbar.update()
    if not hasattr(s, 'passage') or s.passage is None:
        s.passage = cts.getPassage(s)
    if s.passage is None:
        print(f'Failed: {s}')

### Parse with CLTK

In [10]:
pbar = NotebookPBar(max=len(speeches))
for s in speeches:
    pbar.update()
    if not hasattr(s.passage, 'cltk') or s.passage.cltk is None:
        s.passage.runCltkPipeline(remove_punct=True)
    if s.passage.cltk is None:
        print(f'Failed: {s}')

In [ ]:


foo = {}
bar = {}

ids = [i for i,w in enumerate(s.passage.cltk.words) if w.upos != 'PUNCT']

for id_index in range(len(ids)-1):
    i = ids[id_index]
    j = ids[id_index+1]
    lem_i = s.passage.cltk.words[i].lemma
    lem_j = s.passage.cltk.words[j].lemma    
    key = tuple(sorted([lem_i, lem_j]))
    foo.setdefault(key, [])
    foo[key].append((i, j))

In [ ]:
foo